Gesture Controlled Robotic Manipulator (2 DoF) Using Computer Vision 

In [ ]:
from cvzone.HandTrackingModule import HandDetector
from cvzone.SerialModule import SerialObject
from cvzone import putTextRect
import cv2
import numpy as np 


# Initialize the webcam to capture video
# The '2' indicates the third camera connected to your computer; '0' would usually refer to the built-in camera
cap = cv2.VideoCapture(0)

# Initialize the HandDetector class with the given parameters
detector = HandDetector(staticMode=False, maxHands=1, modelComplexity=1, detectionCon=0.5, minTrackCon=0.7)

arduino = SerialObject("COM7", digits=3)

# Continuously get frames from the webcam
while True:
    # Capture each frame from the webcam
    # 'success' will be True if the frame is successfully captured, 'img' will contain the frame
    success, img = cap.read()

    # Find hands in the current frame
    # The 'draw' parameter draws landmarks and hand outlines on the image if set to True
    # The 'flipType' parameter flips the image, making it easier for some detections
    hands, img = detector.findHands(img, draw=True, flipType=True)
    
    H,W,_ = img.shape
   # print(W,H)

    # Check if any hands are detected
    if hands:
        # Information for the first hand detected
        hand1 = hands[0]  # Get the first hand detected
        lmList1 = hand1["lmList"]  # List of 21 landmarks for the first hand
        bbox1 = hand1["bbox"]  # Bounding box around the first hand (x,y,w,h coordinates)
        center1 = hand1['center']  # Center coordinates of the first hand
        handType1 = hand1["type"]  # Type of the first hand ("Left" or "Right")

        # Count the number of fingers up for the first hand
        #fingers1 = detector.fingersUp(hand1)
        #print(f'H1 = {fingers1.count(1)}', end=" ")  # Print the count of fingers that are up

        # Calculate distance between specific landmarks on the first hand and draw it on the image
        #length, info, img = detector.findDistance(lmList1[4][0:2], lmList1[8][0:2], img, color=(255, 0, 255), scale=10)
        
        #print(center1)
        cx1, cy1 = center1
        cv2.rectangle(img, (int(W*0.2),int(0.2*H)), (int(W*0.8),int(0.8*H)), (25,0,51),3)

        servox = int(np.interp(cx1,[W*0.2,W*0.8],[0,180]))
        servoy = int(np.interp(cy1,[H*0.2,H*0.8],[180,0]))

        arduino.sendData([servox,servoy])
        putTextRect(img, f'servox: {int(servox)}', pos=(int(10), int(H*0.05)), scale=1.3, thickness=2, colorR=(0,0,210), offset=8, border=3, colorB=(0,0,0))
        putTextRect(img, f'servoy: {int(servoy)}', pos=(int(500), int(H*0.05)), scale=1.3, thickness=2, colorR=(50,21,0), offset=8, border=3, colorB=(0,0,0))

        cv2.line(img, (cx1,0),(cx1,H), (0,0,255),4)
        cv2.line(img, (0,cy1),(W,cy1), (0,0,255),4)
        cv2.circle(img, (cx1,cy1), 8, (0,255,0), cv2.FILLED)

        # Check if a second hand is detected
        if len(hands) == 2:
            # Information for the second hand
            hand2 = hands[1]
            lmList2 = hand2["lmList"]
            bbox2 = hand2["bbox"]
            center2 = hand2['center']
            handType2 = hand2["type"]

            # Count the number of fingers up for the second hand
            fingers2 = detector.fingersUp(hand2)
            #print(f'H2 = {fingers2.count(1)}', end=" ")

            # Calculate distance between the index fingers of both hands and draw it on the image
            #length, info, img = detector.findDistance(lmList1[8][0:2], lmList2[8][0:2], img, color=(255, 0, 0), scale=10)

        #print(" ")  # New line for better readability of the printed output

    # Display the image in a window
    cv2.imshow("Image", img)

   # Wait for 1 millisecond to check for any user input, keeping the window open
    if cv2.waitKey(1) & 0xFF == ord('q'):
       break
cap.release()
cv2.destroyAllWindows()